In [ ]:
!unzip -O 'cp949' "2021 빅콘테스트_데이터분석분야_퓨처스리그_홍수ZERO_데이터_210803.zip"

Archive:  2021 빅콘테스트_데이터분석분야_퓨처스리그_홍수ZERO_데이터_210803.zip
   creating: 2021 빅콘테스트_데이터분석분야_퓨처스리그_홍수ZERO_데이터_210803/01_제공데이터/
  inflating: 2021 빅콘테스트_데이터분석분야_퓨처스리그_홍수ZERO_데이터_210803/01_제공데이터/2021 빅콘테스트_데이터분석분야_퓨처스리그_홍수ZERO_댐유입량,강우,수위데이터_210803.xlsx  
   creating: 2021 빅콘테스트_데이터분석분야_퓨처스리그_홍수ZERO_데이터_210803/02_평가데이터/
  inflating: 2021 빅콘테스트_데이터분석분야_퓨처스리그_홍수ZERO_데이터_210803/02_평가데이터/2021 빅콘테스트_데이터분석분야_퓨처스리그_홍수ZERO_평가데이터_210803.xlsx  


In [97]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime,timedelta

In [98]:
## 공통

data=pd.read_excel('2021 빅콘테스트_데이터분석분야_퓨처스리그_홍수ZERO_댐유입량,강우,수위데이터_210803.xlsx')

cols=['홍수사상번호','연','월','일','시간','유입량']

for i in range(1,7):
    prefix=f'데이터집단{i}_'
    cols.append(prefix+'유역평균강수')
    cols.append(prefix+'강우(A지역)')
    cols.append(prefix+'강우(B지역)')
    cols.append(prefix+'강우(C지역)')
    cols.append(prefix+'강우(D지역)')
    cols.append(prefix+'수위(E지역)')
    cols.append(prefix+'수위(B지역)')

data=data.drop(0).reset_index(drop=True)
data.columns=cols

for idx,col in enumerate(cols):
    if idx<5:
        data[col]=data[col].astype(int)
    else:
        data[col]=data[col].astype(float)

##Model import

In [99]:
import xgboost as xgb
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1, n_estimator=2000,
                alpha = 10)

#시도

##시도0

In [47]:
## 시도0

train=data.copy()

In [48]:
Y_train=train['유입량']
Y_train=Y_train.iloc[:2891]
train=train.drop(columns=['유입량'], axis=1)
train=train.iloc[:2891,:]

In [49]:
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(train, Y_train, test_size=0.15, random_state=42)

In [50]:
print(x_train.shape,x_valid.shape)

(2457, 47) (434, 47)


In [51]:
from sklearn.metrics import mean_squared_error
xg_reg.fit(x_train,y_train)

y_pred = xg_reg.predict(x_valid)
print(mean_squared_error(y_valid, y_pred, squared=False))

[16:44:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
298.08046765796246


##시도1

In [28]:
## 시도1

train=data.copy()
column=['홍수사상번호','연','월','일','시간','유입량','1유역평균강수','1강우A','1강우B','1강우C','1강우D','1수위E','1수위D','2유역평균강수','2강우A','2강우B','2강우C','2강우D','2수위E','2수위D','3유역평균강수','3강우A','3강우B','3강우C','3강우D','3수위E','3수위D','4유역평균강수','4강우A','4강우B','4강우C','4강우D','4수위E','4수위D','5유역평균강수','5강우A','5강우B','5강우C','5강우D','5수위E','5수위D','6유역평균강수','6강우A','6강우B','6강우C','6강우D','6수위E','6수위D']
train.columns=column

In [27]:
## PCA

Y_train=train['유입량']
Y_train=Y_train.iloc[:2891]
train=train.drop(columns=['유입량'], axis=1)

new_train=train.iloc[:2891,:]
new_test=train.iloc[2891:,:]

from sklearn.decomposition import PCA

lpca = PCA(n_components=30)
train_pca = lpca.fit_transform(new_train)
test_pca=lpca.transform(new_test)

In [11]:
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(train_pca, Y_train, test_size=0.15, random_state=42)

In [19]:
print(x_train.shape,x_valid.shape)

(2457, 30) (434, 30)


In [14]:
from sklearn.metrics import mean_squared_error
xg_reg.fit(x_train,y_train)

y_pred = xg_reg.predict(x_valid)
print(mean_squared_error(y_valid, y_pred, squared=False))

[16:30:42] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
484.72272601053123


##시도2

In [35]:
## 시도2

train=data.copy()

In [36]:
from scipy.cluster.hierarchy import linkage,fcluster

feature=train.drop(['유입량','연','월','일','시간','홍수사상번호'],axis=1)
Z=linkage(feature,'ward')

predict=fcluster(Z,3,criterion='maxclust')
feature['cluster']=predict

## feature가 전체 데이터.

In [38]:
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(feature.iloc[:2891], train['유입량'].iloc[:2891], test_size=0.15, random_state=42)

In [39]:
print(x_train.shape,x_valid.shape)

(2457, 43) (434, 43)


In [40]:
xg_reg.fit(x_train,y_train)

y_pred = xg_reg.predict(x_valid)
print(mean_squared_error(y_valid, y_pred, squared=False))

[16:40:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
341.46550242856495


##시도3

In [52]:
## 시도3

train=data.copy()

In [53]:
def outlier_iqr(data):
  q1, q3 = np.percentile(data, [25, 75])
  iqr = q3-q1
  lower = q1- (iqr*1.5)
  upper = q3 + (iqr*1.5)

  return np.where((data > upper) | (data < lower))

In [54]:
outlier_1 = outlier_iqr(train.loc[train['홍수사상번호']==1].유입량)[0]
outlier_12 = outlier_iqr(train.loc[train['홍수사상번호']==12].유입량)[0]
outlier_15 = outlier_iqr(train.loc[train['홍수사상번호']==15].유입량)[0]
outlier_16 = outlier_iqr(train.loc[train['홍수사상번호']==16].유입량)[0]
outlier_20 = outlier_iqr(train.loc[train['홍수사상번호']==20].유입량)[0]
outlier_22 = outlier_iqr(train.loc[train['홍수사상번호']==22].유입량)[0]

In [55]:
outlier_index = np.concatenate([outlier_1, outlier_12, outlier_15, outlier_16, outlier_20, outlier_22], axis=0)

outlier_index = set(outlier_index)

for i in outlier_index:
  train = train.drop(index=i, axis = 0)

In [56]:
# 시간데이터 삭제

train = train.drop(['연', '월', '일', '시간'], axis=1)

In [57]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2944 entries, 0 to 3050
Data columns (total 44 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   홍수사상번호          2944 non-null   int64  
 1   유입량             2784 non-null   float64
 2   데이터집단1_유역평균강수   2944 non-null   float64
 3   데이터집단1_강우(A지역)  2944 non-null   float64
 4   데이터집단1_강우(B지역)  2944 non-null   float64
 5   데이터집단1_강우(C지역)  2944 non-null   float64
 6   데이터집단1_강우(D지역)  2944 non-null   float64
 7   데이터집단1_수위(E지역)  2944 non-null   float64
 8   데이터집단1_수위(B지역)  2944 non-null   float64
 9   데이터집단2_유역평균강수   2944 non-null   float64
 10  데이터집단2_강우(A지역)  2944 non-null   float64
 11  데이터집단2_강우(B지역)  2944 non-null   float64
 12  데이터집단2_강우(C지역)  2944 non-null   float64
 13  데이터집단2_강우(D지역)  2944 non-null   float64
 14  데이터집단2_수위(E지역)  2944 non-null   float64
 15  데이터집단2_수위(B지역)  2944 non-null   float64
 16  데이터집단3_유역평균강수   2944 non-null   float64
 17  데이터집단3_강우(A지역)  2944 non-null   f

In [58]:
Y_train=train['유입량'].iloc[:2784]
train=train.drop(columns=['유입량'], axis=1).iloc[:2784]

In [60]:
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(train, Y_train, test_size=0.15, random_state=42)

In [61]:
xg_reg.fit(x_train,y_train)

y_pred = xg_reg.predict(x_valid)
print(mean_squared_error(y_valid, y_pred, squared=False))

[16:48:12] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
378.5914040090182


##시도4

In [100]:
## 시도4

## 원래 데이터 data 살려놓기 sub_data로.
sub_data=data.copy()

In [101]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error 
from sklearn.model_selection import train_test_split    
        
from datetime import datetime,timedelta

data['time']=data.apply(lambda row:datetime(year=int(row['연']),month=int(row['월']),day=int(row['일'])),axis=1)
data['time']=data.apply(lambda row:row['time']+timedelta(hours=row['시간']),axis=1)

In [102]:
## 빅콘테스트 수정사항
data.loc[2199,'홍수사상번호']=20
data=data.sort_values(by=['홍수사상번호','time']).reset_index(drop=True)

test=data.iloc[2891:]

submit=test[['홍수사상번호','연','월','일','시간']]
submit['유입량']=0

data=data.drop(['연','월','일','시간'],axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [103]:
## lag 변수
lag_3=data.set_index('time').groupby('홍수사상번호')[[col for col in data.columns if '강우' in col]].shift(3).reset_index().fillna(method='bfill')

lag_3.columns=['time']+['lag3_'+col for col in lag_3.columns[1:]]

data=pd.merge(data,lag_3,on='time',how='left')
data=data.drop(['time'],axis=1)

In [105]:
## 정규화
## 나중에 예측값 내고. (pred+mean)*std 하기.

from sklearn.preprocessing import StandardScaler

for col in data.columns[1:]:
    if col=='유입량':
        mean=data[col].mean()
        std=data[col].std()
        data[col]=(data[col]-mean)/std
        continue
    ss=StandardScaler()
    data[col]=ss.fit_transform(data[col].values.reshape(-1,1))
data.head()

,홍수사상번호,유입량,데이터집단1_유역평균강수,데이터집단1_강우(A지역),데이터집단1_강우(B지역),데이터집단1_강우(C지역),데이터집단1_강우(D지역),데이터집단1_수위(E지역),데이터집단1_수위(B지역),데이터집단2_유역평균강수,데이터집단2_강우(A지역),데이터집단2_강우(B지역),데이터집단2_강우(C지역),데이터집단2_강우(D지역),데이터집단2_수위(E지역),데이터집단2_수위(B지역),데이터집단3_유역평균강수,데이터집단3_강우(A지역),데이터집단3_강우(B지역),데이터집단3_강우(C지역),데이터집단3_강우(D지역),데이터집단3_수위(E지역),데이터집단3_수위(B지역),데이터집단4_유역평균강수,데이터집단4_강우(A지역),데이터집단4_강우(B지역),데이터집단4_강우(C지역),데이터집단4_강우(D지역),데이터집단4_수위(E지역),데이터집단4_수위(B지역),데이터집단5_유역평균강수,데이터집단5_강우(A지역),데이터집단5_강우(B지역),데이터집단5_강우(C지역),데이터집단5_강우(D지역),데이터집단5_수위(E지역),데이터집단5_수위(B지역),데이터집단6_유역평균강수,데이터집단6_강우(A지역),데이터집단6_강우(B지역),데이터집단6_강우(C지역),데이터집단6_강우(D지역),데이터집단6_수위(E지역),데이터집단6_수위(B지역),lag3_데이터집단1_강우(A지역),lag3_데이터집단1_강우(B지역),lag3_데이터집단1_강우(C지역),lag3_데이터집단1_강우(D지역),lag3_데이터집단2_강우(A지역),lag3_데이터집단2_강우(B지역),lag3_데이터집단2_강우(C지역),lag3_데이터집단2_강우(D지역),lag3_데이터집단3_강우(A지역),lag3_데이터집단3_강우(B지역),lag3_데이터집단3_강우(C지역),lag3_데이터집단3_강우(D지역),lag3_데이터집단4_강우(A지역),lag3_데이터집단4_강우(B지역),lag3_데이터집단4_강우(C지역),lag3_데이터집단4_강우(D지역),lag3_데이터집단5_강우(A지역),lag3_데이터집단5_강우(B지역),lag3_데이터집단5_강우(C지역),lag3_데이터집단5_강우(D지역),lag3_데이터집단6_강우(A지역),lag3_데이터집단6_강우(B지역),lag3_데이터집단6_강우(C지역),lag3_데이터집단6_강우(D지역)
0,1,-0.714002,-1.036870,-0.988808,-0.905344,-0.712134,-0.808934,-0.852663,-1.480226,-1.047460,-1.017853,-0.898056,-0.708746,-0.808934,-0.852663,-1.496831,-1.047460,-1.017853,-0.842268,-0.714176,-0.815826,-0.852663,-1.502656,-1.054742,-1.032332,-0.842268,-0.828119,-0.839624,-0.852663,-1.438416,-1.039463,-1.032332,-0.842268,-0.828119,-0.839624,-0.852663,-1.447929,-0.989485,-1.032332,-0.850690,-0.690847,-0.819482,-0.852663,-1.462364,-0.964683,-0.881456,-0.693639,-0.787613,-0.993108,-0.874678,-0.690951,-0.787613,-0.993108,-0.821229,-0.696998,-0.794331,-1.007268,-0.821229,-0.807466,-0.817654,-1.007268,-0.821229,-0.807466,-0.817654,-1.007268,-0.829359,-0.676979,-0.798248
1,1,-0.701236,-1.038555,-0.988808,-0.889561,-0.712134,-0.808934,-0.856874,-1.481258,-1.045774,-1.017853,-0.882157,-0.708746,-0.808934,-0.856874,-1.495455,-1.045774,-1.017853,-0.825489,-0.714176,-0.815826,-0.856874,-1.501280,-1.039552,-1.032332,-0.825489,-0.789880,-0.804437,-0.856874,-1.439596,-1.024050,-1.032332,-0.825489,-0.789880,-0.804437,-0.856874,-1.449912,-0.973335,-1.032332,-0.834046,-0.646145,-0.783537,-0.856874,-1.464014,-0.964683,-0.881456,-0.693639,-0.787613,-0.993108,-0.874678,-0.690951,-0.787613,-0.993108,-0.821229,-0.696998,-0.794331,-1.007268,-0.821229,-0.807466,-0.817654,-1.007268,-0.821229,-0.807466,-0.817654,-1.007268,-0.829359,-0.676979,-0.798248
2,1,-0.685436,-1.036870,-0.988808,-0.873778,-0.712134,-0.808934,-0.856874,-1.482289,-1.030598,-1.017853,-0.866258,-0.708746,-0.808934,-0.856874,-1.494080,-1.030598,-1.017853,-0.808710,-0.692176,-0.815826,-0.856874,-1.499903,-1.024363,-1.032332,-0.808710,-0.789880,-0.786844,-0.856874,-1.440777,-1.008636,-1.032332,-0.808710,-0.789880,-0.786844,-0.856874,-1.451894,-0.957185,-1.032332,-0.817403,-0.646145,-0.765564,-0.856874,-1.465664,-0.964683,-0.881456,-0.693639,-0.787613,-0.993108,-0.874678,-0.690951,-0.787613,-0.993108,-0.821229,-0.696998,-0.794331,-1.007268,-0.821229,-0.807466,-0.817654,-1.007268,-0.821229,-0.807466,-0.817654,-1.007268,-0.829359,-0.676979,-0.798248
3,1,-0.661883,-1.021706,-0.988808,-0.857995,-0.712134,-0.808934,-0.856874,-1.482289,-1.015422,-1.017853,-0.850359,-0.686761,-0.808934,-0.856874,-1.492705,-1.015422,-1.017853,-0.791931,-0.648175,-0.780323,-0.856874,-1.498527,-0.972043,-1.005092,-0.791931,-0.694285,-0.734063,-0.856874,-1.441958,-0.955544,-1.005092,-0.791931,-0.694285,-0.734063,-0.856874,-1.454537,-0.901556,-1.005092,-0.800759,-0.534390,-0.711646,-0.856874,-1.466489,-0.964683,-0.881456,-0.693639,-0.787613,-0.993108,-0.874678,-0.690951,-0.787613,-0.993108,-0.821229,-0.696998,-0.794331,-1.007268,-0.821229,-0.807466,-0.817654,-1.007268,-0.821229,-0.807466,-0.817654,-1.007268,-0.829359,-0.676979,-0.798248
4,1,-0.624313,-1.006542,-0.988808,-0.826429,-0.690437,-0.773422,-0.856874,-1.482289,-0.963151,-0.990423,-0.818560,-0.642791,-0.773422,-0.856874,-1.491329,-0.963151,-0.990423,-0.758373,-0.6

In [106]:
data.shape

(3051, 68)

In [108]:
Y_train=data['유입량'].iloc[:2891]
train=data.drop(columns=['유입량'], axis=1).iloc[:2891]

In [109]:
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(train, Y_train, test_size=0.15, random_state=42)

In [110]:
xg_reg.fit(x_train,y_train)

y_pred = xg_reg.predict(x_valid)
print(mean_squared_error(y_valid, y_pred, squared=False))

[17:32:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.1423741957450201


In [120]:
## 나중에 예측값 내고. (pred+mean)*std 하기.
scaled_pred=(y_pred*std)+mean
scaled_pred=[x if x>0 else 0 for x in scaled_pred]
scaled_test_label=(y_valid*std)+mean

from sklearn.metrics import mean_squared_error
print(mean_squared_error(scaled_test_label,scaled_pred, squared=False))

310.63061191037787
